## Task 1
**We are predicting number of boundaries and number of extras in the final match. As you will see in task 2 and 3, we are assuming that predictions we are making is the ground truth and so final will be between India and South Africa in Narendra Modi Stadium,Ahmedabad**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('deliveries.csv')
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.1,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.2,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.3,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.4,England,New Zealand,DJ Malan,JM Bairstow,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.5,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df2 = df[['match_id','innings','venue','batting_team','bowling_team','runs_off_bat']]

In [ ]:
filtered_df = df2[df2['runs_off_bat'].isin([4, 6])]

In [ ]:
filtered_df['Total_Boundaries'] = filtered_df.groupby(['match_id','innings'])['runs_off_bat'].transform('count')


<ipython-input-5-18971c4c2b08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Total_Boundaries'] = filtered_df.groupby(['match_id','innings'])['runs_off_bat'].transform('count')


In [ ]:
filtered_df_unique = filtered_df.drop_duplicates(subset=['match_id','innings'])

In [ ]:
filtered_df_unique = filtered_df_unique.drop(columns=['match_id','innings','runs_off_bat'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
lst = ['venue','batting_team','bowling_team']
for i in lst:
    filtered_df_unique[i] = le.fit_transform(filtered_df_unique[i])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print(le_name_mapping)

{'Arun Jaitley Stadium, Delhi': 0, 'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow': 1, 'Eden Gardens, Kolkata': 2, 'Himachal Pradesh Cricket Association Stadium, Dharamsala': 3, 'M Chinnaswamy Stadium, Bengaluru': 4, 'MA Chidambaram Stadium, Chepauk, Chennai': 5, 'Maharashtra Cricket Association Stadium, Pune': 6, 'Narendra Modi Stadium, Ahmedabad': 7, 'Rajiv Gandhi International Stadium, Uppal, Hyderabad': 8, 'Wankhede Stadium, Mumbai': 9}
{'Afghanistan': 0, 'Australia': 1, 'Bangladesh': 2, 'England': 3, 'India': 4, 'Netherlands': 5, 'New Zealand': 6, 'Pakistan': 7, 'South Africa': 8, 'Sri Lanka': 9}
{'Afghanistan': 0, 'Australia': 1, 'Bangladesh': 2, 'England': 3, 'India': 4, 'Netherlands': 5, 'New Zealand': 6, 'Pakistan': 7, 'South Africa': 8, 'Sri Lanka': 9}


In [ ]:
from sklearn.model_selection import train_test_split
X = filtered_df_unique.drop(columns='Total_Boundaries')
y = filtered_df_unique['Total_Boundaries']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)

In [ ]:
import xgboost as xgb
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score


param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'n_estimators': [100, 200, 300]
}

xg_reg = xgb.XGBRegressor(objective='reg:squarederror')

grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f'Best Hyperparameters: {best_params}')

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Root Mean Squared Error on Test Set:", np.sqrt(mse))

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Root Mean Squared Error on Test Set: 11.805805502896684


In [ ]:
# 4 represents India
# 8 represents South Africa
# 7 in venue represents Narendra Modi Stadium, Ahmedabad
new_data1 = pd.DataFrame({'venue': [7], 'batting_team': [4], 'bowling_team': [8]})
new_data2 = pd.DataFrame({'venue': [7], 'batting_team': [8], 'bowling_team': [4]})

y_pred1 = best_model.predict(new_data1)
print("Boundaries when India bats:", y_pred1)
y_pred2 = best_model.predict(new_data2)
print("Boundaries when South Africa bats:",y_pred2)
print("Total Boundaries in whole match:", y_pred1+y_pred2)

Boundaries when India bats: [28.48503]
Boundaries when South Africa bats: [30.156712]
Total Boundaries in whole match: [58.64174]


In [ ]:
from sklearn.linear_model import Ridge
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
alphas = [0.001, 0.01, 0.1, 1.0, 10.0,3.0]
param_grid = {'alpha': alphas}

grid_search = GridSearchCV(Ridge(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_alpha = grid_search.best_params_['alpha']
best_ridge = grid_search.best_estimator_
# 4 represents India
# 8 represents South Africa
# 7 in venue represents Narendra Modi Stadium, Ahmedabad
new_data1 = pd.DataFrame({'venue': [7], 'batting_team': [4], 'bowling_team': [8]})
new_data2 = pd.DataFrame({'venue': [7], 'batting_team': [8], 'bowling_team': [4]})

y_pred1 = best_ridge.predict(new_data1)
print("Boundaries when India bats:", y_pred1)
y_pred2 = best_ridge.predict(new_data2)
print("Boundaries when South Africa bats:",y_pred2)
print("Total Boundaries in whole match:", y_pred1+y_pred2)

Boundaries when India bats: [29.67489908]
Boundaries when South Africa bats: [31.55646656]
Total Boundaries in whole match: [61.23136564]


## Observation: There is very small difference between Ridge regression and Xgboost but as we have very little number of datapoints it is good to compare both the models

## Now we will try to find the number of runs going to be conceded in extras.

In [ ]:
data = pd.read_csv('deliveries.csv')
check = data[['match_id','innings','venue','batting_team','bowling_team','extras']]
check['total_extras'] = check.groupby(['match_id','innings'])['extras'].transform('sum')
df_unique = check.drop_duplicates(subset=['match_id','innings'])
df_unique = df_unique.drop(columns=['match_id'])
df_unique = df_unique.drop(columns=['extras'])

<ipython-input-14-5d4f1cbeaa7a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['total_extras'] = check.groupby(['match_id','innings'])['extras'].transform('sum')


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
lst = ['venue','batting_team','bowling_team']
for i in lst:
    df_unique[i] = le.fit_transform(df_unique[i])
df_unique

,innings,venue,batting_team,bowling_team,total_extras
0,1,7,3,6,6
304,2,7,6,3,8
525,1,8,7,5,9
827,2,8,5,7,9
1082,1,3,0,2,8
...,...,...,...,...,...
16103,2,6,0,9,10
16383,1,2,2,7,9
16659,2,2,7,2,4
16856,1,6,8,6,12


In [ ]:
X = df_unique.drop(columns='total_extras')
y = df_unique['total_extras']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE) on the test set
mse = mean_squared_error(y_test, y_pred)
print("Root Mean Squared Error on Test Set:", np.sqrt(mse))

Root Mean Squared Error on Test Set: 5.663687135930702


In [ ]:
new_data1 = pd.DataFrame({'innings':[1],'venue': [7], 'batting_team': [4], 'bowling_team': [8]})
new_data2 = pd.DataFrame({'innings':[1],'venue': [7], 'batting_team': [8], 'bowling_team': [4]})

y_pred1 = model.predict(new_data1)
print("Extras when India bats:", y_pred1)
y_pred2 = model.predict(new_data2)
print("Extras when South Africa bats:",y_pred2)
print("Total Extras in whole match:", y_pred1+y_pred2)

Extras when India bats: [14.170511]
Extras when South Africa bats: [6.7626677]
Total Extras in whole match: [20.933178]


## Task 2 & 3
**Top 4 teams are India,South Africa,New Zealand and Australia. Based on this information we will predict who will get qualified for the final and who will win the world cup as well**

In [ ]:
data = pd.read_csv('odi_Matches_Data.csv')
data.head()

,ODI Match No,Match ID,Match Name,Series ID,Series Name,Match Date,Match Format,Team1 ID,Team1 Name,Team1 Captain,...,Umpire 2,Match Referee,Toss Winner,Toss Winner Choice,Match Winner,Match Result Text,MOM Player,Team1 Playing 11,Team2 Playing 11,Debut Players
0,488,65425,Australia Vs New Zealand 4Th Match,60879.0,"Benson & Hedges World Series Cup Australia, Ne...",1988-01-07,ODI,2.0,Australia,1572.0,...,RC Bailhache,NaN,Australia,bat,Australia,Australia won by 6 runs,1795.0,"['1767', '1793', '1754', '1572', '1871', '1795...","['1550', '1863', '1861', '1669', '1698', '1846...",[]
1,492,65428,New Zealand Vs Sri Lanka 7Th Match,60879.0,"Benson & Hedges World Series Cup Australia, Ne...",1988-01-12,ODI,5.0,New Zealand,1698.0,...,SG Randell,NaN,Sri Lanka,bowl,Sri Lanka,Sri Lanka won by 4 wickets (with 21 balls rema...,1810.0,"['1777', '1550', '1698', '1669', '1861', '1846...","['1810', '1864', '1789', '1762', '1666', '1664...",[]
2,495,65431,Australia Vs New Zealand 10Th Match,60879.0,"Benson & Hedges World Series Cup Australia, Ne...",1988-01-17,ODI,5.0,New Zealand,1698.0,...,AR Crafter,NaN,Australia,bowl,Australia,Australia won by 5 wickets (with 64 balls rema...,1795.0,"['1550', '1863', '1698', '1669', '1407', '1790...","['1793', '1767', '1773', '1754', '1871', '1795...",['1790']
3,496,65432,Australia Vs Sri Lanka 11Th Match,60879.0,"Benson & Hedges World Series Cup Australia, Ne...",1988-01-19,ODI,8.0,Sri Lanka,1664.0,...,TA Prue,NaN,Australia,bowl,Australia,Australia won by 3 wickets (with 3 balls remai...,1871.0,"['1810', '1864', '1789', '1753', '1762', '1666...","['1767', '1793', '1754', '1871', '1795', '1572...",['1753']
4,508,64326,New Zealand Vs England 3Rd Odi,60882.0,England tour of New Zealand - 1988 (1987/88),1988-03-16,ODI,1.0,England,1543.0,...,SJ Woodward,NaN,New Zealand,bowl,New Zealand,New Zealand won by 7 wickets (with 21 balls re...,1550.0,"['1758', '1629', '1770', '1850', '1543', '1865...","['1550', '1883', '1863', '1882', '1777', '1918...",['1883']


In [ ]:
team_names = [
    'Australia',
    'New Zealand',
    'Sri Lanka',
    'England',
    'India',
    'Pakistan',
    'Bangladesh',
    'South Africa',
    'Netherlands',
    'Afghanistan'
]
df = pd.DataFrame()

for team in team_names:
    team_data = data[data['Team1 Name'] == team]
    df = pd.concat([df, team_data], ignore_index=True)

team_names = [
    'Australia',
    'New Zealand',
    'Sri Lanka',
    'England',
    'India',
    'Pakistan',
    'Bangladesh',
    'South Africa',
    'Netherlands',
    'Afghanistan'
]

df1 = pd.DataFrame()

for team in team_names:
    team_data = df[df['Team2 Name'] == team]
    df1 = pd.concat([df1, team_data], ignore_index=True)


In [ ]:
dataset = df1[['Team1 Name','Team2 Name','Match Venue (Stadium)','Match Venue (City)','Match Venue (Country)','Match Winner']]

In [ ]:
dataset['Winner'] = pd.NA

# Use a conditional statement to set the values in 'Match Winner Encoded' based on the winner
dataset.loc[dataset['Match Winner'] == dataset['Team1 Name'], 'Winner'] = 0
dataset.loc[dataset['Match Winner'] == dataset['Team2 Name'], 'Winner'] = 1

<ipython-input-30-949ae5bf3b91>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Winner'] = pd.NA


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(dataset['Match Venue (City)'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)
dataset['Match Venue (City)'] = le.transform(dataset['Match Venue (City)'])

{'Abu Dhabi': 0, 'Adelaide': 1, 'Ahmedabad': 2, 'Amritsar': 3, 'Amstelveen': 4, 'Auckland': 5, 'Ballarat': 6, 'Basseterre': 7, 'Belfast': 8, 'Bengaluru': 9, 'Benoni': 10, 'Birmingham': 11, 'Bloemfontein': 12, 'Bogra': 13, 'Bridgetown': 14, 'Brighton': 15, 'Brisbane': 16, 'Bristol': 17, 'Bulawayo': 18, 'Cairns': 19, 'Canberra': 20, 'Canterbury': 21, 'Cape Town': 22, 'Cardiff': 23, 'Centurion': 24, 'Chandigarh': 25, 'Chattogram': 26, 'Chelmsford': 27, 'Chennai': 28, 'Chester-le-Street': 29, 'Christchurch': 30, 'Colombo': 31, 'Cuttack': 32, 'Dambulla': 33, 'Darwin': 34, 'Delhi': 35, 'Derby': 36, 'Dhaka': 37, 'Dharamsala': 38, 'Doha': 39, 'Dubai': 40, 'Dublin': 41, 'Dunedin': 42, 'Durban': 43, 'East London': 44, 'Faisalabad': 45, 'Faridabad': 46, 'Fatullah': 47, 'Galle': 48, 'Glasgow': 49, 'Gqeberha': 50, 'Gros Islet': 51, 'Gujranwala': 52, 'Guwahati': 53, 'Gwalior': 54, 'Hambantota': 55, 'Hamilton': 56, 'Harare': 57, 'Hobart': 58, 'Hyderabad': 59, 'Indore': 60, 'Jaipur': 61, 'Jalandhar': 

<ipython-input-31-5bfb6ac03f11>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Match Venue (City)'] = le.transform(dataset['Match Venue (City)'])


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(dataset['Match Venue (Country)'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)
dataset['Match Venue (Country)'] = le.transform(dataset['Match Venue (Country)'])

{'Australia': 0, 'Bangladesh': 1, 'Canada': 2, 'England': 3, 'India': 4, 'Ireland': 5, 'Kenya': 6, 'Malaysia': 7, 'Morocco': 8, 'Netherlands': 9, 'New Zealand': 10, 'Pakistan': 11, 'Qatar': 12, 'Scotland': 13, 'Singapore': 14, 'South Africa': 15, 'Sri Lanka': 16, 'United Arab Emirates': 17, 'West Indies': 18, 'Zimbabwe': 19}


<ipython-input-32-3e27eee61cf0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Match Venue (Country)'] = le.transform(dataset['Match Venue (Country)'])


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(dataset['Match Venue (Stadium)'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)
dataset['Match Venue (Stadium)'] = le.transform(dataset['Match Venue (Stadium)'])

{'AMI Stadium': 0, 'Adelaide Oval': 1, 'Andhra Cricket Association-Visakhapatnam District Cricket Association Stadium': 2, 'Arbab Niaz Stadium': 3, 'Arun Jaitley Stadium': 4, 'Asgiriya Stadium': 5, 'Ayub National Stadium': 6, 'Bangabandhu National Stadium': 7, 'Barabati Stadium': 8, 'Barsapara Cricket Stadium': 9, 'Basin Reserve': 10, 'Bay Oval': 11, 'Beausejour Stadium': 12, 'Bellerive Oval': 13, 'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium': 14, 'Boland Bank Park': 15, 'Boland Park': 16, 'Brabourne Stadium': 17, 'Brisbane Cricket Ground': 18, 'Buffalo Park': 19, 'Bundaberg Rum Stadium': 20, 'Burlton Park': 21, 'Captain Roop Singh Stadium': 22, 'Carisbrook': 23, "Cazaly's Stadium": 24, 'Centurion Park': 25, 'Chevrolet Park': 26, 'Chittagong Divisional Stadium': 27, 'Chittagong Stadium': 28, 'City Oval': 29, 'Civil Service Cricket Club': 30, 'Clontarf Cricket Club Ground': 31, 'County Ground': 32, 'Crusaders Ground': 33, 'Davies Park': 34, 'De Beers Diamond Oval': 35, 

<ipython-input-33-c712e597466f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Match Venue (Stadium)'] = le.transform(dataset['Match Venue (Stadium)'])


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(dataset['Team1 Name'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)
dataset['Team1 Name'] = le.transform(dataset['Team1 Name'])

{'Afghanistan': 0, 'Australia': 1, 'Bangladesh': 2, 'England': 3, 'India': 4, 'Netherlands': 5, 'New Zealand': 6, 'Pakistan': 7, 'South Africa': 8, 'Sri Lanka': 9}


<ipython-input-34-41e2edba955e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Team1 Name'] = le.transform(dataset['Team1 Name'])


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(dataset['Team2 Name'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)
dataset['Team2 Name'] = le.transform(dataset['Team2 Name'])

{'Afghanistan': 0, 'Australia': 1, 'Bangladesh': 2, 'England': 3, 'India': 4, 'Netherlands': 5, 'New Zealand': 6, 'Pakistan': 7, 'South Africa': 8, 'Sri Lanka': 9}


<ipython-input-35-f9afe566c734>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Team2 Name'] = le.transform(dataset['Team2 Name'])


In [ ]:
dataset = dataset.dropna()

In [ ]:
X = dataset.drop(columns=['Match Winner','Winner','Match Venue (Stadium)','Match Venue (Country)'],axis=1)
y = dataset['Winner']
y=y.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
X_train.shape

(1911, 3)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a Sequential model
model = Sequential()
# Add the input layer with input shape (30, 1)
model.add(Dense(units=64, activation='relu', input_shape=(3, )))
# Add the first hidden layer
model.add(Dense(units=32, activation='relu'))
# Add the second hidden layer
# model.add(Dense(units=32, activation='relu'))
# Add the output layer with sigmoid activation for binary classification
model.add(Dense(units=1, activation='sigmoid'))
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Display the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                256       
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2369 (9.25 KB)
Trainable params: 2369 (9.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Predict probabilities (scores) for each class
predictions = model.predict(X_test)

Epoch 1/100
60/60 [==============================] - 0s 6ms/step - loss: 0.6577 - accuracy: 0.5887
Epoch 2/100
60/60 [==============================] - 0s 5ms/step - loss: 0.6533 - accuracy: 0.5882
Epoch 3/100
60/60 [==============================] - 0s 8ms/step - loss: 0.6537 - accuracy: 0.5882
Epoch 4/100
60/60 [==============================] - 0s 3ms/step - loss: 0.6532 - accuracy: 0.5965
Epoch 5/100
60/60 [==============================] - 0s 3ms/step - loss: 0.6536 - accuracy: 0.5939
Epoch 6/100
60/60 [==============================] - 0s 3ms/step - loss: 0.6528 - accuracy: 0.5840
Epoch 7/100
60/60 [==============================] - 0s 3ms/step - loss: 0.6525 - accuracy: 0.5829
Epoch 8/100
60/60 [==============================] - 0s 2ms/step - loss: 0.6526 - accuracy: 0.5835
Epoch 9/100
60/60 [==============================] - 0s 2ms/step - loss: 0.6523 - accuracy: 0.5877
Epoch 10/100
60/60 [==============================] - 0s 3ms/step - loss: 0.6531 - accuracy: 0.5924
Epoch 11/

In [ ]:
accuracy = model.evaluate(X_test, y_test)[1]
print(f'Test Accuracy: {accuracy}')

20/20 [==============================] - 0s 2ms/step - loss: 0.7424 - accuracy: 0.5212
Test Accuracy: 0.6511930871009827


## We have also trained RandomForest Regressor and LinearSVC at accuracy of 54% and 61% but as this deep learning model outperforms them, we will be passing onto our data in this model only.

## Here we are encoding before giving as input based on the mapping we have printed earlier and sf1,sf2 and final are matches if the result is near 1 then team2 wins and if result is near 0 then team1 wins

In [ ]:
sf1 = pd.Series([6,4,85], index=X.columns)
sf2 = pd.Series([8,1,71], index=X.columns)
df_copy = X.copy()
df_copy.drop(df_copy.index, inplace=True)
df_copy = df_copy.append(sf1, ignore_index=True)
df_copy = df_copy.append(sf2, ignore_index=True)

<ipython-input-61-fa967cfadb0a>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_copy = df_copy.append(sf1, ignore_index=True)
<ipython-input-61-fa967cfadb0a>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_copy = df_copy.append(sf2, ignore_index=True)


In [ ]:
result1 = model.predict(df_copy)
print(result1)

1/1 [==============================] - 0s 26ms/step
[[0.52964145]
 [0.47731328]]


## Based on this from semi final-1 India will win and from semi final-2 South Africa will win and face each other in finals.

In [ ]:
final = pd.Series([8,4,2],index=X.columns)
df_copy.drop(df_copy.index, inplace=True)
df_copy = df_copy.append(final, ignore_index=True)

<ipython-input-63-037aea4cbd4c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_copy = df_copy.append(final, ignore_index=True)


In [ ]:
final_result = model.predict(df_copy)
print(final_result)

1/1 [==============================] - 0s 22ms/step
[[0.95191264]]


## Based on our dataset and model prediction, India will win the ICC World Cup 2023

## Here in the example we have reduced input features to make it simple and a pickle file is also generated of the same model but to increase accuracy we can make it complex.

In [ ]:
df = pd.read_csv('man_odi_data_2.csv')
df.head()

,Match Date,Team1 Name,Team1 Captain,Team2 Name,Team2 Captain,Match Venue (Stadium),Match Venue (City),Match Venue (Country),Toss Winner,Toss Winner Choice,Match Winner,Team1 Playing 11,Team2 Playing 11,Debut Players
0,07-01-1988,Australia,1572,New Zealand,1698,Melbourne Cricket Ground,Melbourne,Australia,Australia,bat,Australia,"['1767', '1793', '1754', '1572', '1871', '1795...","['1550', '1863', '1861', '1669', '1698', '1846...",[]
1,12-01-1988,New Zealand,1698,Sri Lanka,1664,Bellerive Oval,Hobart,Australia,Sri Lanka,bowl,Sri Lanka,"['1777', '1550', '1698', '1669', '1861', '1846...","['1810', '1864', '1789', '1762', '1666', '1664...",[]
2,17-01-1988,New Zealand,1698,Australia,1572,Brisbane Cricket Ground,Brisbane,Australia,Australia,bowl,Australia,"['1550', '1863', '1698', '1669', '1407', '1790...","['1793', '1767', '1773', '1754', '1871', '1795...",['1790']
3,19-01-1988,Sri Lanka,1664,Australia,1572,Sydney Cricket Ground,Sydney,Australia,Australia,bowl,Australia,"['1810', '1864', '1789', '1753', '1762', '1666...","['1767', '1793', '1754', '1871', '1795', '1572...",['1753']
4,21-10-1988,India,1491,West Indies,1433,Sharjah Cricket Association Stadium,Sharjah,United Arab Emirates,India,bat,West Indies,"['1653', '1639', '1353', '1491', '1774', '1568...","['1433', '1553', '1744', '1873', '1697', '1654...",[]


In [ ]:
player_list = set([])
for cell in df['Team1 Playing 11']:
    lst = cell.strip("[]").replace("'", "").replace(' ', '').split(',')
    lst = [int(x) for x in lst]
    for player in lst:
        player_list.add(player)
player_list = list(player_list)

In [ ]:
player_list = set([])
for cell in df['Team2 Playing 11']:
    lst = cell.strip("[]").replace("'", "").replace(' ', '').split(',')
    lst = [int(x) for x in lst]
    for player in lst:
        player_list.add(player)
player_list = list(player_list)

In [ ]:
team_1_players_df = pd.DataFrame(columns=['team1_P1','team1_P2','team1_P3','team1_P4','team1_P5','team1_P6','team1_P7','team1_P8','team1_P9','team1_P10','team1_P11'])
for x in df['Team1 Playing 11']:
    arr = np.array(x.replace('[', '').replace(']', '').replace("'", "").split(', '))
    arr = arr.astype('int')
    team_1_players_df.loc[len(team_1_players_df)] = arr

In [ ]:
team_2_players_df = pd.DataFrame(columns=['team2_P1','team2_P2','team2_P3','team2_P4','team2_P5','team2_P6','team2_P7','team2_P8','team2_P9','team2_P10','team2_P11'])
for x in df['Team2 Playing 11']:
    arr = np.array(x.replace('[', '').replace(']', '').replace("'", "").split(', '))
    arr = arr.astype('int')
    team_2_players_df.loc[len(team_2_players_df)] = arr

In [ ]:
df = pd.concat([df, team_1_players_df, team_2_players_df], axis=1)

In [ ]:
winner = []
for match_win, team_one in zip(df['Match Winner'], df['Team1 Name']):
    if match_win == team_one:
        winner.append(0)
    else:
        winner.append(1)

In [ ]:
df['match_winner'] = winner

In [ ]:
df = df.drop(columns=['Match Winner', 'Toss Winner','Debut Players','Team1 Playing 11','Team2 Playing 11','Match Date'], axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
# df = pd.get_dummies(data=df, columns=['Team1 Name', 'Team1 Captain', 'Team2 Name', 'Team2 Captain', 'Match Venue (Stadium)', 'Match Venue (City)',
#                                        'Match Venue (Country)', 'Toss Winner Choice'])
le = LabelEncoder()
lst = ['team1_P1','team1_P2','team1_P3','team1_P4','team1_P5','team1_P6','team1_P7','team1_P8','team1_P9','team1_P10','team1_P11','team2_P1','team2_P2','team2_P3','team2_P4','team2_P5','team2_P6','team2_P7','team2_P8','team2_P9','team2_P10','team2_P11','Team1 Name', 'Team1 Captain', 'Team2 Name', 'Team2 Captain', 'Match Venue (Stadium)', 'Match Venue (City)','Match Venue (Country)', 'Toss Winner Choice']
for i in lst:
    df[i] = le.fit_transform(df[i])

In [ ]:
from sklearn.model_selection import train_test_split
x = df.drop(columns=['match_winner'], axis=1)
y = df['match_winner']

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a Sequential model
model = Sequential()
# Add the input layer with input shape (30, 1)
model.add(Dense(units=64, activation='relu', input_shape=(30, )))
# Add the first hidden layer
model.add(Dense(units=32, activation='relu'))
# Add the second hidden layer
model.add(Dense(units=16, activation='relu'))
# Add the output layer with sigmoid activation for binary classification
model.add(Dense(units=1, activation='sigmoid'))
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Display the model summary
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 64)                1984      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 16)                528       
                                                                 
 dense_18 (Dense)            (None, 1)                 17        
                                                                 
Total params: 4609 (18.00 KB)
Trainable params: 4609 (18.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=32)

# Predict probabilities (scores) for each class
predictions = model.predict(x_test)

Epoch 1/50
67/67 [==============================] - 1s 2ms/step - loss: 16.5805 - accuracy: 0.4927
Epoch 2/50
67/67 [==============================] - 0s 2ms/step - loss: 1.2635 - accuracy: 0.5045
Epoch 3/50
67/67 [==============================] - 0s 2ms/step - loss: 0.9531 - accuracy: 0.5176
Epoch 4/50
67/67 [==============================] - 0s 2ms/step - loss: 0.8501 - accuracy: 0.5467
Epoch 5/50
67/67 [==============================] - 0s 3ms/step - loss: 0.7865 - accuracy: 0.5571
Epoch 6/50
67/67 [==============================] - 0s 3ms/step - loss: 0.7797 - accuracy: 0.5547
Epoch 7/50
67/67 [==============================] - 0s 4ms/step - loss: 0.7414 - accuracy: 0.5669
Epoch 8/50
67/67 [==============================] - 0s 3ms/step - loss: 0.7067 - accuracy: 0.5702
Epoch 9/50
67/67 [==============================] - 0s 3ms/step - loss: 0.7124 - accuracy: 0.5829
Epoch 10/50
67/67 [==============================] - 0s 3ms/step - loss: 0.7507 - accuracy: 0.5514
Epoch 11/50
67/67 

In [ ]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy}')

23/23 [==============================] - 0s 3ms/step - loss: 0.7850 - accuracy: 0.5394
Test Accuracy: 0.7394366383552551


## We have used Xgboost regressor and it gave us accuracy of 58% without tuning and 64% with tuning

### When it comes to predict the playing 11 of any team in cricket it is very difficult because some things are not quantifiable but is crucial in determining the squad, so we have used Naive Bayes to calculate the probabilities of all the team players based on how many matches have they played in this world cup out of total 9 matches.

In [ ]:
data = pd.read_csv('players - Sheet1.csv')

In [ ]:
data = data.fillna(0)
data['Label'] = data['Label'].astype('int')

In [ ]:
data

,Player Name,Match1,Match2,Match3,Match4,Match5,Match6,Match7,Match8,Match9,Label
0,Rohit Sharma (c),1,1,1,1,1,1,1,1,1,0
1,Hardik Pandya (vc),1,1,1,1,0,0,0,0,0,0
2,Shubman Gill,0,0,1,1,1,1,1,1,1,0
3,Virat Kohli,1,1,1,1,1,1,1,1,1,0
4,Shreyas Iyer,1,1,1,1,1,1,1,1,1,0
5,KL Rahul,1,1,1,1,1,1,1,1,1,0
6,Ravindra Jadeja,1,1,1,1,1,1,1,1,1,0
7,Shardul Thakur,0,1,1,1,0,0,0,0,0,0
8,Jasprit Bumrah,1,1,1,1,1,1,1,1,1,0
9,Mohammed Siraj,1,1,1,1,1,1,1,1,1,0


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Player Name'] = le.fit_transform(data['Player Name'])
X = data.drop(columns=['Label','Match9'])
y = data['Match9']
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(X, y)

MultinomialNB()

In [ ]:
X_test = data.drop(columns=['Label','Match9'])
y_pred = nb_model.predict(X_test)
probability_positive = nb_model.predict_proba(X_test)[:, 1]

In [ ]:
probability_positive

array([0.92120104, 0.24028045, 0.97991464, 0.91548932, 0.91895997,
       0.92758801, 0.9223002 , 0.36208387, 0.92860498, 0.92445649,
       0.9265577 , 0.99082886, 0.54183121, 0.38271199, 0.98965204])

In [1]:
print("The Playing 11 for India will be as follows: ")
print("Rohit Sharma, Shubman Gill, Virat Kohli, Shreyas Iyer, KL Rahul, Ravindra Jadeja, Jasprit Bumrah, Mohammed Siraj, Mohammed Shami,Kuldeep Yadav and Suryakumar Yadav")

The Playing 11 for India will be as follows: 
Rohit Sharma, Shubman Gill, Virat Kohli, Shreyas Iyer, KL Rahul, Ravindra Jadeja, Jasprit Bumrah, Mohammed Siraj, Mohammed Shami,Kuldeep Yadav and Suryakumar Yadav


In [4]:
data = pd.read_csv('Players - Sheet1.csv')
data = data.fillna(0)
data['Label'] = data['Label'].astype('int')
data

,Player Name,Match1,Match2,Match3,Match4,Match5,Match6,Match7,Match8,Match9,Label
0,Temba Bavuma,1,1,1,0,0,1,1,1,1,0
1,Reeza Hendricks,0,0,0,1,1,0,0,0,0,0
2,Aidam Markram,1,1,1,1,1,1,1,1,1,0
3,David Miller,1,1,1,1,1,1,1,1,1,0
4,Rassie van der Dussen,1,1,1,1,1,1,1,1,1,0
5,Marco Jansen,1,1,1,1,1,1,1,1,0,0
6,Andile Phehlukwayo,0,0,0,0,0,0,0,0,1,0
7,Quinton de kock,1,1,1,1,1,1,1,1,1,0
8,Heinrich Klaasen,1,1,1,1,1,1,1,1,1,0
9,Keshav Maharaj,1,1,1,1,1,1,1,1,1,0


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Player Name'] = le.fit_transform(data['Player Name'])
X = data.drop(columns=['Label','Match9'])
y = data['Match9']
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(X, y)

MultinomialNB()

In [6]:
X_test = data.drop(columns=['Label','Match9'])
y_pred = nb_model.predict(X_test)
probability_positive = nb_model.predict_proba(X_test)[:, 1]

In [7]:
probability_positive

array([0.31449678, 0.0330995 , 0.99394183, 0.98675245, 0.68072818,
       0.82444875, 0.64948337, 0.75986729, 0.97127783, 0.93884734,
       0.87677463, 0.9401213 , 0.04724805, 0.95972549, 0.14518934])

In [8]:
print("The Playing 11 for South Africa will be as follows: ")
print("Temba Bavuma, Aidam Markram, David Miller, Rassie Van der Dussen,Heinrich Klassen, Quinton de Kock,Keshav Maharaj, Kagiso Rabada, Lungi Ngidi,Gerald Coeztee,Marco Janson")

The Playing 11 for South Africa will be as follows: 
Temba Bavuma, Aidam Markram, David Miller, Rassie Van der Dussen,Heinrich Klassen, Quinton de Kock,Keshav Maharaj, Kagiso Rabada, Lungi Ngidi,Gerald Coeztee,Marco Janson
